# Shuffle the passages in the repllama-train-tevatron.

In [3]:
import json
import random

#设置随机种子
random.seed(0)
with open('/root/paddlejob/workspace/env_run/data/msmarco-pass/repllama-train-tevatron_1.jsonl', 'w', encoding="utf-8") as file:
    #打开你的文件
    with open('/root/paddlejob/workspace/env_run/data/msmarco-pass/repllama-train-tevatron.jsonl', "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            random.shuffle(data['positive_passages'])
            random.shuffle(data['negative_passages'])
            file.write(json.dumps(data)+"\n")

# Process the files of the relevance selection and utility-based selection results. 

In [21]:
import json
import re
with open("results/annotation_relevance_final.jsonl", "w", encoding="utf-8") as file_w:
    with open("results/annotation_relevance.jsonl", "r", encoding="utf-8") as file_r:
        for line in file_r:
            js = json.loads(line)
            numbers = re.findall(r'\[(\d+)\]', js["relevance_output"])
            numbers = [int(number)-1 for number in numbers]
            utility_scores = [0]*len(js["passages_ids"])
            for number in numbers:
                if number >= len(utility_scores):
                    continue
                utility_scores[number]=1
            js["relevance_score"] = utility_scores
            file_w.write(json.dumps(js)+"\n")
        file_r.close()
        

In [ ]:
import json
import re
with open("results/annotation_utiity_final.jsonl", "w", encoding="utf-8") as file_w:
    with open("results/annotation_utility.jsonl", "r", encoding="utf-8") as file_r:
        for line in file_r:
            js = json.loads(line)
            numbers = re.findall(r'\[(\d+)\]', js["utility_output"])
            numbers = [int(number)-1 for number in numbers]
            utility_scores = [0]*len(js["passages_ids"])
            for number in numbers:
                if number >= len(utility_scores):
                    continue
                utility_scores[number]=1
            js["utility_score"] = utility_scores
            file_w.write(json.dumps(js)+"\n")
        file_r.close()
        

# Process the utility ranking results

In [ ]:
import json
import re
def extract_substrings(input_string):
        pattern = re.compile(r'\[\d+\]')
        substrings = pattern.findall(input_string)
        extracted_string = ''.join(substrings)
        return extracted_string

def clean_response(response: str):
    response = extract_substrings(response)
    new_response = ''
    for c in response:
        if not c.isdigit():
            new_response += ' '
        else:
            new_response += c
    new_response = new_response.strip()
    return new_response


def remove_duplicate(response):
    new_response = []
    for c in response:
        if c not in new_response:
            new_response.append(c)
    return new_response
with open("results/Qwen-utility-ranking_final.jsonl", "w", encoding="utf-8") as file_w:
    with open("results/Qwen-utility-ranking.jsonl", "r", encoding="utf-8") as file2:
        for line in file2:
            js = json.loads(line)
            relevace_ranking = js["utility_ranking"]
            response = clean_response(relevace_ranking)
            response = [int(x) - 1 for x in response.split()]
            response = remove_duplicate(response)
            response = [i for i in response if i >= 0 and i < len(js["passages_ids"])]
            js["ranking_score"] = response
            ranked_passage_ids = []
            for i in response:
                ranked_passage_ids.append(js["passages_ids"][i])
            js["ranked_passage_ids"] = ranked_passage_ids
            file_w.write(json.dumps(js)+"\n")
